# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298418714548                   -0.85    5.0
  2   -8.300182771914       -2.75       -1.26    1.2
  3   -8.300430292352       -3.61       -1.88    2.8
  4   -8.300460866982       -4.51       -2.74    1.8
  5   -8.300463842772       -5.53       -3.07    4.5
  6   -8.300464236870       -6.40       -3.25    5.9
  7   -8.300464451947       -6.67       -3.41    1.6
  8   -8.300464553324       -6.99       -3.56    2.0
  9   -8.300464621207       -7.17       -3.76    1.5
 10   -8.300464634959       -7.86       -3.92    1.4
 11   -8.300464643150       -8.09       -4.26    2.0


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.65996051672                   -0.70    6.6
  2   -16.66413787938       -2.38       -1.14    1.0
  3   -16.66886286722       -2.33       -1.88    1.6
  4   -16.67927285147       -1.98       -2.50    4.5
  5   -16.67928514165       -4.91       -2.79    3.6
  6   -16.67928584166       -6.15       -3.10    2.8
  7   -16.67928596616       -6.90       -3.19    2.1
  8   -16.67928618344       -6.66       -3.59    1.1
  9   -16.67928621651       -7.48       -3.99    2.1
 10   -16.67928622319       -8.18       -4.42    2.9


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32752936895                   -0.56    7.5
  2   -33.33451672015       -2.16       -1.00    1.8
  3   -33.33595498374       -2.84       -1.72    5.5
  4   -33.33613330251       -3.75       -2.61    4.2
  5   -33.33679922278       -3.18       -2.24    7.1
  6   -33.33681938396       -4.70       -2.28    1.5
  7   -33.33694311568       -3.91       -3.40    2.4
  8   -33.33694316575       -7.30       -3.37    4.1
  9   -33.33694369374       -6.28       -3.62    1.4
 10   -33.33694392402       -6.64       -4.62    2.2


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298650557634                   -0.85    5.2
  2   -8.300287812437       -2.79       -1.59    1.2
  3   -8.300438027423       -3.82       -2.66    2.2
  4   -8.300428019305   +   -5.00       -2.47    5.9
  5   -8.300464348982       -4.44       -3.36    1.0
  6   -8.300464587547       -6.62       -3.79    3.6
  7   -8.300464642003       -7.26       -4.37    2.1


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32627036432                   -0.56    8.4
  2   -33.27772664609   +   -1.31       -1.25    1.9
  3   +13.04479729261   +    1.67       -0.23    6.8
  4   -33.30738578935        1.67       -1.68    5.9
  5   -33.23104683799   +   -1.12       -1.31    3.8
  6   -33.29290993846       -1.21       -1.72    2.8
  7   -32.84052653478   +   -0.34       -1.22    4.5
  8   -33.33522209494       -0.31       -2.31    3.9
  9   -33.33666515015       -2.84       -2.71    3.0
 10   -33.33675624658       -4.04       -2.70    3.0
 11   -33.33691308190       -3.80       -3.09    2.0
 12   -33.33693852112       -4.59       -3.36    2.0
 13   -33.33694148719       -5.53       -3.60    3.2
 14   -33.33694341461       -5.72       -3.97    2.1
 15   -33.33694295079   +   -6.33       -4.05    2.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.